## DIA 033: Versionado de la API con Blueprints en Flask

En este día, implementaremos el versionado de la API utilizando Blueprints en Flask. El versionado es una práctica fundamental para evolucionar la API sin romper la compatibilidad con los clientes existentes. Con Blueprints, puedes definir diferentes versiones de los endpoints y registrar cada uno con un prefijo de URL distinto (por ejemplo, /api/v1 y /api/v2). De esta forma, los clientes pueden optar por seguir utilizando la versión antigua mientras se introducen mejoras y cambios en la nueva versión.

En este ejemplo, crearemos dos versiones del endpoint /predict:

Versión 1 (v1): Retorna una respuesta simple (por ejemplo, siempre el dígito 5 con 90% de probabilidad).
Versión 2 (v2): Retorna una respuesta mejorada (por ejemplo, siempre el dígito 7 con 95% de probabilidad y un mensaje adicional).
Además, se mantendrá un endpoint común para el login, el cual se utilizará para obtener el token JWT necesario para acceder a los endpoints protegidos en ambas versiones. También se incluirá un endpoint de health check para facilitar la supervisión del estado de la aplicación.

Código Completo (api.py)
python
Copiar
from flask import Flask, Blueprint, request, jsonify, render_template, url_for
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from flask_bcrypt import Bcrypt
from flask_mail import Mail, Message
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
import os
import logging

# Configuración básica y variables de entorno
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'your_secret_key')
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'your_jwt_secret_key')
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL', 'sqlite:///app.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Inicialización de extensiones
db = SQLAlchemy(app)
migrate = Migrate(app, db)
bcrypt = Bcrypt(app)
mail = Mail(app)
jwt = JWTManager(app)
limiter = Limiter(app, key_func=get_remote_address, default_limits=["200 per day", "50 per hour"])

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Definición de un modelo de usuario simple (para ejemplo)
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    # Otros campos se omiten por simplicidad

# ---------------------------
# Blueprint para API v1
# ---------------------------
api_v1 = Blueprint('api_v1', __name__)

@api_v1.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("100 per day")
def predict_v1():
    """
    Versión 1 del endpoint predict.
    Se espera una imagen y se devuelve una respuesta simple.
    """
    if 'file' not in request.files:
        return jsonify({"error": "No se encontró el archivo"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No se seleccionó ningún archivo"}), 400

    # Simulación simple: siempre retorna dígito 5 con 90% de probabilidad
    result = {"prediccion": 5, "probabilidad": 0.90, "version": "v1"}
    logger.info("v1: Predicción realizada.")
    return jsonify(result), 200

# ---------------------------
# Blueprint para API v2
# ---------------------------
api_v2 = Blueprint('api_v2', __name__)

@api_v2.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("150 per day")
def predict_v2():
    """
    Versión 2 del endpoint predict.
    Se espera una imagen y se devuelve una respuesta con información adicional.
    """
    if 'file' not in request.files:
        return jsonify({"error": "No se encontró el archivo"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No se seleccionó ningún archivo"}), 400

    # Simulación mejorada: retorna dígito 7 con 95% de probabilidad y añade un campo extra
    result = {"prediccion": 7, "probabilidad": 0.95, "version": "v2", "mensaje": "Predicción mejorada"}
    logger.info("v2: Predicción realizada con mejoras.")
    return jsonify(result), 200

# ---------------------------
# Endpoint de Login (común para ambas versiones)
# ---------------------------
@app.route('/login', methods=['POST'])
def login():
    """
    Endpoint para que los usuarios inicien sesión y obtengan un token JWT.
    """
    data = request.get_json()
    username = data.get('username', None)
    password = data.get('password', None)
    
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400
    
    # Aquí se debería validar contra la base de datos; se asume que el usuario existe para el ejemplo
    access_token = create_access_token(identity=username)
    logger.info(f"Usuario '{username}' inició sesión.")
    return jsonify(access_token=access_token), 200

# ---------------------------
# Health Check Endpoint (opcional)
# ---------------------------
@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok"}), 200

# ---------------------------
# Registro de Blueprints (versionado de la API)
# ---------------------------
app.register_blueprint(api_v1, url_prefix='/api/v1')
app.register_blueprint(api_v2, url_prefix='/api/v2')

# ---------------------------
# Ejecutar la aplicación
# ---------------------------
if __name__ == '__main__':
    app.run(debug=True)
Explicación del Código
Configuración de la Aplicación y Extensiones:
Se configura la aplicación Flask con variables de entorno, y se inicializan extensiones como SQLAlchemy, JWT, Bcrypt, Mail y Limiter. También se configura el logging para registrar eventos y se establece un logger.

Definición de Blueprints para Versionado:
Se crean dos Blueprints, api_v1 y api_v2, cada uno con su propio endpoint /predict:

Versión 1 (v1): Devuelve siempre el dígito 5 con una probabilidad del 90%.
Versión 2 (v2): Devuelve siempre el dígito 7 con una probabilidad del 95% y añade un mensaje extra.
Endpoint de Login Común:
Se define un endpoint /login que genera un token JWT para los usuarios. Este endpoint es utilizado por ambas versiones de la API para autenticación.

Health Check:
Se añade un endpoint /health para comprobar el estado de la aplicación.

Registro de Blueprints:
Se registra cada blueprint con un prefijo de URL distinto, permitiendo a los clientes acceder a la versión que deseen mediante rutas como /api/v1/predict y /api/v2/predict.

Ejecución de la Aplicación:
La aplicación se ejecuta en modo debug cuando se inicia directamente.